In [5]:
 using ForwardDiff, GenericLinearAlgebra, LinearAlgebra

In [2]:
using Pkg

In [3]:
Pkg.add("ForwardDiff")

    Updating registry at `~/.julia/registries/General`
┌ Error: Some registries failed to update:
│     — `~/.julia/registries/General` — registry dirty
└ @ Pkg.Registry /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.7/Pkg/src/Registry/Registry.jl:450
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [4]:
Pkg.add("GenericLinearAlgebra")

   Resolving package versions...
    Updating `~/.julia/environments/v1.7/Project.toml`
  [14197337] + GenericLinearAlgebra v0.2.5
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [14197337] + GenericLinearAlgebra v0.2.5
Precompiling project...
  ✓ GenericLinearAlgebra
  1 dependency successfully precompiled in 2 seconds (277 already precompiled, 1 skipped during auto due to previous errors)


## Singular Value Decomposition
$$ dA = \prod_{i < j} |\sigma_i^2 - \sigma_j^2| \prod_i\sigma_i^{m-n}(H'dU)^\wedge(d\Sigma)^\wedge (V'dV)^\wedge $$

In [6]:
function sv_prod(σ)
    n = length(σ)
    abs(prod([σ[i]^2 - σ[j]^2 for i in 1:n, j in 1:n if i<j]))   
end

function check_svd(x)
    H, σ, V = svd(x, full=false)
  
    JU = ForwardDiff.jacobian(t -> svd(t).U, x)
    Jσ = ForwardDiff.jacobian(t -> svd(t).S, x)
    JV = ForwardDiff.jacobian(t -> copy(svd(t).V), x)

    Uk    = kron(Matrix(I, size(x)), H') * JU   
    Vk    = kron(Matrix(I, size(V)), V') * JV
    rowsU = vec((tril(ones(size(x)), -1))) .== 1
    rowsV = vec((tril(ones(size(V)), -1))) .== 1

    Uk = Uk[rowsU, :]
    Vk = Vk[rowsV, :]
    m, n = size(x)
    (abs(det(vcat( Jσ, Uk, Vk))) , 1/(sv_prod(σ)*prod(σ)^(m - n) ))
end

check_svd (generic function with 1 method)

In [11]:
x = randn(3,3)
check_svd(x)

(0.19947988709616224, 0.1994798870961451)